<a href="https://colab.research.google.com/github/gaallmin/Prediction-of-Agricultural-Product-prices/blob/yeji/%ED%8C%8C%EC%83%9D%EB%B3%80%EC%88%98_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Dacon

Mounted at /content/drive/
/content/drive/MyDrive/Dacon


In [29]:
import pandas as pd
import re

# CSV 파일 불러오기
train_file_path = './train/train.csv'
market_file_path = './train/meta/TRAIN_전국도매_2018-2021.csv'
sanjae_file_path = './train/meta/TRAIN_산지공판장_2018-2021.csv'

train_df = pd.read_csv(train_file_path)
market_df = pd.read_csv(market_file_path)
sanjae_df = pd.read_csv(sanjae_file_path)

# '시장명'이 "전국도매시장"인 경우만 필터링
market_df = market_df[market_df['시장명'].str.contains('전국도매시장')]

# '공판장명'이 "전국농협공판장"인 경우만 필터링
sanjae_df = sanjae_df[sanjae_df['공판장명'].str.contains('전국농협공판장')]

sanjae_df.rename(columns = {'등급명': '등급'}, inplace = True)

# 컬럼명 변경 (도매 데이터의 컬럼들에 '(도매)', 산지 데이터의 컬럼들에 '(산지)' 추가)
market_columns = market_df.columns.difference(['시점', '품목명', '품종명'])  # 키 컬럼 제외
market_renamed = {col: col + ' (도매)' for col in market_columns}

sanjae_columns = sanjae_df.columns.difference(['시점', '품목명', '품종명', '등급'])  # 키 컬럼 제외
sanjae_renamed = {col: col + ' (산지)' for col in sanjae_columns}

# 컬럼명 변경 적용
market_df.rename(columns={**market_renamed}, inplace=True)
sanjae_df.rename(columns={**sanjae_renamed}, inplace=True)

# '시점', '품목명', '품종명' 기준으로 market_df와 left join, '등급명'은 train_df에만 존재
merged_df = pd.merge(train_df, market_df, how='left', on=['시점', '품목명', '품종명'])

# '시점', '품목명', '품종명', '등급명' 기준으로 sanjae_df와 left join
merged_sanjae_df = pd.merge(merged_df, sanjae_df, how='left', on=['시점', '품목명', '품종명', '등급'])

# '거래단위'에서 'kg'을 제외한 숫자만 추출하여 새로운 컬럼 '거래단위_숫자' 생성
merged_sanjae_df['거래단위_숫자'] = merged_sanjae_df['거래단위'].apply(lambda x: float(re.sub(r'[^0-9.]', ' ', x)))

# '거래단위'에서 '5톤트럭'과 '8톤트럭' 처리
def process_truck_units(unit):
    if '5톤트럭' in unit:
        return 5000
    elif '8톤트럭' in unit:
        return 8000
    else:
        return float(re.sub(r'[^0-9.]', '', unit))  # 그 외의 경우 숫자만 추출

merged_sanjae_df['거래단위_숫자'] = merged_sanjae_df['거래단위'].apply(process_truck_units)

# '평균가(원/kg)'와 '거래단위_숫자'를 곱해서 '거래금액(도매)' 및 '거래금액(산지)' 컬럼에 저장
merged_sanjae_df['거래금액(도매)'] = merged_sanjae_df['평균가(원/kg) (도매)'] * merged_sanjae_df['거래단위_숫자']
merged_sanjae_df['거래금액(산지)'] = merged_sanjae_df['평균가(원/kg) (산지)'] * merged_sanjae_df['거래단위_숫자']

# '거래금액(도매)'와 '거래금액(산지)'에서 값이 없거나 0인 경우 0으로 처리
merged_sanjae_df['거래금액(도매)'] = merged_sanjae_df['거래금액(도매)'].fillna(0)
merged_sanjae_df['거래금액(도매)'] = merged_sanjae_df['거래금액(도매)'].apply(lambda x: 0 if x == 0 or pd.isna(x) else x)

merged_sanjae_df['거래금액(산지)'] = merged_sanjae_df['거래금액(산지)'].fillna(0)
merged_sanjae_df['거래금액(산지)'] = merged_sanjae_df['거래금액(산지)'].apply(lambda x: 0 if x == 0 or pd.isna(x) else x)

# 결과 확인
print(merged_sanjae_df[['시점', '품목명', '품종명', '등급', '거래단위', '거래단위_숫자', '평균가(원/kg) (도매)', '거래금액(도매)', '평균가(원/kg) (산지)', '거래금액(산지)']].head())

         시점  품목명 품종명  등급   거래단위  거래단위_숫자  평균가(원/kg) (도매)  거래금액(도매)  \
0  201801상순  건고추  화건  상품  30 kg     30.0             NaN       0.0   
1  201801중순  건고추  화건  상품  30 kg     30.0             NaN       0.0   
2  201801하순  건고추  화건  상품  30 kg     30.0             NaN       0.0   
3  201802상순  건고추  화건  상품  30 kg     30.0             NaN       0.0   
4  201802중순  건고추  화건  상품  30 kg     30.0             NaN       0.0   

   평균가(원/kg) (산지)  거래금액(산지)  
0             NaN       0.0  
1             NaN       0.0  
2             NaN       0.0  
3             NaN       0.0  
4             NaN       0.0  


<ipython-input-29-eabfa57526ae>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_df.rename(columns={**market_renamed}, inplace=True)


In [30]:
# 수정된 데이터를 새로운 CSV 파일로 저장 (UTF-8-SIG 인코딩 사용)
output_file_path = './merged_with_transaction_amount.csv'
merged_sanjae_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')